# Core tracking API by examples

This notebook illustrates the use of core APIs with a simple example.

In [ ]:
try:
    import google.colab

    %pip install -q --upgrade laptrack matplotlib spacy flask pandas
    # upgrade packages to avoid pip warnings if the notebook is run in colab
except:
    %pip install -q --upgrade laptrack matplotlib pandas

**Note**: Restart the runtime when you are on Google Colab and you see an error in matplotlib.

## Importing packages

`laptrack.LapTrack` is the core object for tracking.

In [ ]:
from os import path
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from laptrack import LapTrack
from laptrack import datasets

## Loading data

Load the example point coordinates from a CSV file.

`spots_df` has columns `["frame", "position_x", "position_y"]` 
(can be arbitrary, and the column names for tracking will be specified later).

In [ ]:
spots_df = datasets.simple_tracks()
spots_df["frame"] = spots_df["frame"] + 10
display(spots_df.head())

## Tracking

### Initializing LapTrack object

First, initialize the `LapTrack` object with parameters. 

In [ ]:
max_distance = 15
lt = LapTrack(
    metric="sqeuclidean",  # The similarity metric for particles. See `scipy.spatial.distance.cdist` for allowed values.
    splitting_metric="sqeuclidean",
    merging_metric="sqeuclidean",
    # the square of the cutoff distance for the "sqeuclidean" metric
    cutoff=max_distance**2,
    splitting_cutoff=max_distance**2,  # or False for non-splitting case
    merging_cutoff=max_distance**2,  # or False for non-merging case
)

### Example 1: using `predict_dataframe` to track pandas DataFrame coordinates

`predict_dataframe` is the easiest option when you have the coordinate data in pandas DataFrame.


In [ ]:
track_df, split_df, merge_df = lt.predict_dataframe(
    spots_df,
    coordinate_cols=[
        "position_x",
        "position_y",
    ],  # the column names for the coordinates
    frame_col="frame",  # the column name for the frame (default "frame")
)

`track_df` is the original dataframe with additional columns "track_id" and "tree_id".

The track_id is a unique id for each track segments without branches. A new id is assigned when a splitting and merging occured. 

The tree_id is a unique id for each "clonal" tracks sharing the same ancestor.

In [ ]:
track_df.head()

In [ ]:
keys = ["frame", "position_x", "position_y", "track_id", "tree_id"]
display(track_df[keys].head())

`split_df` is a dataframe for splitting events with the following columns:
- "parent_track_id" : the track id of the parent
- "child_track_id" : the track id of the parent

In [ ]:
display(split_df)

`merge_df` is a dataframe for merging events with the following columns:
- "parent_track_id" : the track id of the parent
- "child_track_id" : the track id of the parent

In [ ]:
display(merge_df)

We can display tracks in `napari` as follows:

In [ ]:
# Does not work in Colab
#
# import napari
# v = napari.Viewer()
# v.add_points(spots_df[["frame", "position_x", "position_y"]])
# track_df2 = track_df.reset_index()
# v.add_tracks(track_df2[["track_id", "frame", "position_x", "position_y"]])

Plotting tracks in `matplotlib`

In [ ]:
plt.figure(figsize=(3, 3))
frames = track_df["frame"]
frame_range = [frames.min(), frames.max()]
k1, k2 = "position_y", "position_x"
keys = [k1, k2]


def get_track_end(track_id, first=True):
    df = track_df[track_df["track_id"] == track_id].sort_index(level="frame")
    return df.iloc[0 if first else -1][keys].values


for track_id, grp in track_df.groupby("track_id"):
    df = grp.reset_index().sort_values("frame")
    plt.scatter(df[k1], df[k2], c=df["frame"], vmin=frame_range[0], vmax=frame_range[1])
    for i in range(len(df) - 1):
        pos1 = df.iloc[i][keys].values
        pos2 = df.iloc[i + 1][keys].values
        plt.plot([pos1[0], pos2[0]], [pos1[1], pos2[1]], "-k")
    for _, row in list(split_df.iterrows()) + list(merge_df.iterrows()):
        pos1 = get_track_end(row["parent_track_id"], first=False)
        pos2 = get_track_end(row["child_track_id"], first=True)
        plt.plot([pos1[0], pos2[0]], [pos1[1], pos2[1]], "-k")


plt.xticks([])
plt.yticks([])

### Example 2: using `predict` to track frame-wise-organized coordinates to make networkx tree

`predict_dataframe` is a thin wrapper of the `predict` function, the core tracking function of the `LapTrack` object. 

One can directly use this function with the input of the frame-wise coordinate list and the output of networkx `DiGraph` object representing the lineage tree.

In [ ]:
frame_min = spots_df["frame"].min()
frame_max = spots_df["frame"].max()
coords = []
for i in range(frame_min, frame_max):
    df = spots_df[spots_df["frame"] == i]
    coords.append(df[["position_x", "position_y"]].values)

The input variable `coords` should be organized as the frame-wise list of the point coordinates.

The coordinate dimension is `(particle, dimension)`.

In [ ]:
display(coords[:5])

`predict` function generates a networkx `DiGraph` object from the coordinates

In [ ]:
track_tree = lt.predict(
    coords,
)

The returned `track_tree` is the connection between spots, represented as ((frame1,index1), (frame2,index2)) ...

For example, `((0, 0), (1, 1))` means the connection between `[178.41257464, 185.18866074]` and `[185.1758993 , 185.18866074]` in this example.

In [ ]:
for edge in list(track_tree.edges())[:5]:
    print(edge)